# WARNING
Clean notebook before pushing to Github. 
In the menu: Cell -> All Output -> Clear

In [ ]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import mne

# Loading & cropping
EEG files are huge (~10GB/night) so we crop one hour for the tests.

In [ ]:
path_to_file = '../data/raw/EEG/Nathalie-78_20171118_123017.mff'

In [ ]:
raw = mne.io.read_raw_egi(path_to_file,
                          montage='GSN-HydroCel-256',
                          preload=True)

In [ ]:
raw.crop(tmin=3600, tmax=3600*2)
raw.filter(0.1, 50., fir_design='firwin')
raw.set_eeg_reference('average', projection=True)
raw.plot(block=True, lowpass=40, highpass=0.1)
plt.show()

In [ ]:
raw.save('../data/derived/eeg_example_raw.fif', overwrite=True)

# Preprocessing

In [ ]:
raw = mne.io.read_raw_fif('../data/derived/eeg_example_raw.fif', preload=True)

In [ ]:
from autoreject import AutoReject, compute_thresholds
from functools import partial
import numpy as np
np.random.seed(42)

In [ ]:
events_ar = mne.event.make_fixed_length_events(
            raw, id=9999, start=0, stop=None, duration=1.0,
            first_samp=True)
epochs_ar = mne.epochs.Epochs(raw, events_ar, tmin=0, tmax=1.0,
                              baseline=None, preload=True)
epochs_ar.decimate(2)

In [ ]:
thresh_func = partial(compute_thresholds, random_state=42, n_jobs=1)
ar = AutoReject(thresh_func=thresh_func, verbose='tqdm')

index = np.random.choice(np.arange(len(epochs_ar)),
                         size=int(np.floor(len(epochs_ar) * 0.1)),
                         replace=False)
ar.fit(epochs_ar[index])

In [ ]:
epochs_clean = ar.transform(epochs_ar)

print("{:.2f}% epochs rejected (N={})".format(
      epochs_clean.drop_log_stats(), len(epochs_clean)))

In [ ]:
epochs_clean.save('../data/derived/cleaned_epo.fif')

## Check epochs

In [ ]:
epochs_clean = mne.read_epochs('../data/derived/cleaned_epo.fif')

In [ ]:
plt.figure(figsize=[20, 10])
epochs_clean.plot_psd_topomap()
plt.show()

In [ ]:
plt.figure(figsize=[20, 10])
epochs_clean.plot_psd()
plt.show()

## Source reconstruction

In [ ]:
src = mne.setup_source_space('nathalie', subjects_dir='../data/raw/MRI/', spacing='oct6') 
mne.write_source_spaces('../data/derived/nathalie-oct6-src.fif', src)

In [ ]:
%%bash
mne watershed_bem -s nathalie

In [ ]:
model = mne.make_bem_model('nathalie', subjects_dir='../data/raw/MRI/')
mne.write_bem_surfaces('../data/derived/nathalie-bem.fif', model) 
bem_sol = mne.make_bem_solution(model)  
mne.write_bem_solution('../data/derived/nathalie-bem-sol.fif', bem_sol) 